In [1]:
from VariationalAutoDecoder import VariationalAutoDecoder as VAD
from VAD_Trainer import VAD_Trainer
import utils
from evaluate import evaluate_model
import torch
import torch.optim as optim
import torch.nn as nn
import csv
import time
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Create DataLoaders

In [2]:
train_ds, train_dl, test_ds, test_dl = utils.create_dataloaders(data_path="dataset" ,batch_size=64)

## Train Auto Decoder

In [3]:
import torch.nn as nn

architectures = [
    # Architecture 1: Balanced Depth with Dropout
    {
        "mu_net": nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256)
        )
    },

    # Architecture 2: Deeper and Wider Network with Batch Normalization
    {
        "mu_net": nn.Sequential(
            nn.Linear(256, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(256, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256)
        )
    },

    # Architecture 3: Lightweight Architecture with High Dropout
    {
        "mu_net": nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256)
        )
    },

    # Architecture 4: Residual Connections and Dropout
    {
        "mu_net": nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Sequential(
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Dropout(0.25)
            ),
            nn.Linear(512, 256)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Sequential(
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Dropout(0.25)
            ),
            nn.Linear(512, 256)
        )
    },

    # Architecture 5: Small Network with Layer Normalization
    {
        "mu_net": nn.Sequential(
            nn.Linear(256, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 256)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(256, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 256)
        )
    }
]




# latent_dims = [dim for dim in [64, 32, 128, 16, 10] for _ in range(5)]
VADs = [VAD(mu_layers=arch['mu_net'], var_layers=arch['log_var_net']) for arch in architectures]# for _ in range(5)]
# learning_rates = [lr for lr in [0.001, 0.0005, 0.0001, 0.002, 0.005] for _ in range(5)]
trainers = [VAD_Trainer(var_decoder=VADs[i], dataloader=train_dl, latent_dim=256, device=device, lr=1e-3) for i in range(len(VADs))]

In [4]:
# Initialize the results list to hold all the data
num_test_samples = len(test_dl.dataset)

# Create latent parameters and optimizers for each trainer
latents_list = [torch.nn.Parameter(torch.randn(num_test_samples, trainers[i].latent_dim).to(device)) for i in range(5)]# for _ in range(5)]
optimizers = [optim.Adam([latents], lr=1e-3) for latents in latents_list]

# Save results to a CSV file
csv_file_path = 'results_VAD_temp.csv'

# Write header to the CSV file first
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['Index'] + [f'Epoch {i+1} Loss' for i in range(200)] + ['Final Test Loss']
    writer.writerow(header)

# Main training and evaluation loop
for index, trainer in enumerate(trainers):
    start_time = time.time()  # Record the start time
    train_loss = trainer.train(num_epochs=200)  # Train the model
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"Trainer {index} has finished training in {elapsed_time:.2f} seconds.")

    start_time = time.time()  # Record the start time
    test_loss = evaluate_model(model=VADs[index], test_dl=test_dl, opt=optimizers[index], latents=latents_list[index], epochs=100, device=device) 
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"AD {index} has finished test evaluation in {elapsed_time:.2f} seconds.")

    # Prepare the row to be saved
    row = [index] + train_loss + [test_loss]

    # Append results to the CSV file after each iteration
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

print(f"Results saved to {csv_file_path}.")

torch.Size([64, 1, 16, 16])


RuntimeError: shape '[-1, 1, 28, 28]' is invalid for input of size 4194304

In [8]:
print(f"mu is {VADs[0].mu}  ")
print(f"var is {VADs[0].log_var}  ")

mu is tensor([[-0.0075,  0.3937,  0.4571,  ..., -0.4561,  1.1955,  0.5147],
        [ 0.9493,  0.4686, -0.2689,  ..., -0.5194, -0.0215, -0.4969],
        [ 0.5074,  0.0087,  0.3487,  ..., -0.1180, -0.4948, -0.1248],
        ...,
        [ 0.2776, -0.6547, -0.8342,  ..., -0.4688, -0.1061,  0.5185],
        [ 0.4887,  0.7447,  0.2863,  ..., -0.8428,  1.5053, -0.2989],
        [ 0.6870,  0.3778, -0.3577,  ...,  0.1559, -0.9798,  0.2507]],
       device='cuda:0')  
var is tensor([[ 1.8487,  0.9140,  1.2749,  ...,  1.9546,  2.4533,  1.5519],
        [ 2.4893,  0.6447,  3.2543,  ...,  3.9568,  2.7078,  0.4194],
        [ 2.2148,  1.8637,  3.0983,  ...,  1.6641,  2.5564,  1.1311],
        ...,
        [ 1.3752,  1.4912,  1.1600,  ..., -0.6805,  1.9990,  1.8013],
        [ 1.8618,  2.3971,  3.3418,  ...,  1.4460,  1.9874,  1.3415],
        [ 0.1172,  1.9712,  1.6490,  ...,  0.8563,  2.3387,  1.7930]],
       device='cuda:0')  


In [9]:
utils.plot_tsne(train_ds, trainers[0].latents, "tsne0")

<Figure size 800x600 with 0 Axes>

## Sample specific vectors

In [ ]:
arch = nn.Sequential(
        nn.Linear(256, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.25),
        nn.Linear(512, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Linear(1024, 2048),
        nn.BatchNorm1d(2048),
        nn.ReLU(),
        nn.Linear(2048, 4096),
        nn.ReLU(),
        nn.Linear(4096, 784)
    )
decoder = AutoDecoder.AutoDecoder(arch)
trainer = AD_Trainer.AD_Trainer(decoder=decoder, dataloader=train_dl, latent_dim=256, device=device, lr=5e-3)

In [17]:
trainer.train(num_epochs=200)
num_test_samples = len(test_dl.dataset)
latents = torch.nn.Parameter(torch.randn(num_test_samples, trainer.latent_dim).to(device))
opt = optim.Adam([latents], lr=1e-3)

Epoch [1/200], Loss: 0.0718
Epoch [2/200], Loss: 0.0713
Epoch [3/200], Loss: 0.0686
Epoch [4/200], Loss: 0.0682
Epoch [5/200], Loss: 0.0726
Epoch [6/200], Loss: 0.0740
Epoch [7/200], Loss: 0.0714
Epoch [8/200], Loss: 0.0662
Epoch [9/200], Loss: 0.0631
Epoch [10/200], Loss: 0.0636
Epoch [11/200], Loss: 0.0643
Epoch [12/200], Loss: 0.0651
Epoch [13/200], Loss: 0.0648
Epoch [14/200], Loss: 0.0687
Epoch [15/200], Loss: 0.0705
Epoch [16/200], Loss: 0.0666
Epoch [17/200], Loss: 0.0629
Epoch [18/200], Loss: 0.0643
Epoch [19/200], Loss: 0.0640
Epoch [20/200], Loss: 0.0629
Epoch [21/200], Loss: 0.0645
Epoch [22/200], Loss: 0.0647
Epoch [23/200], Loss: 0.0669
Epoch [24/200], Loss: 0.0721
Epoch [25/200], Loss: 0.0681
Epoch [26/200], Loss: 0.0658
Epoch [27/200], Loss: 0.0668
Epoch [28/200], Loss: 0.0650
Epoch [29/200], Loss: 0.0656
Epoch [30/200], Loss: 0.0694
Epoch [31/200], Loss: 0.0635
Epoch [32/200], Loss: 0.0639
Epoch [33/200], Loss: 0.0656
Epoch [34/200], Loss: 0.0624
Epoch [35/200], Loss: 0

In [19]:
test_loss = evaluate_model(model=decoder, test_dl=test_dl, opt=opt, latents=latents, epochs=1000, device=device)
print(f"AD has finished test evaluation with a test loss of {test_loss}.")

AD has finished test evaluation with a test loss of 0.1657548677176237.


In [24]:
# Randomly sample 5 indices from the test dataset
random.seed(6)
sampled_indices = random.sample(range(len(latents)), 5)

# Extract the corresponding vectors (input data) and their labels
sampled_latents = [latents[i] for i in sampled_indices]  # Only selecting input data, not labels

# Convert to a single tensor (optional)
sampled_latents_tensor = torch.stack(sampled_latents)
random_latents_tensor = torch.randn_like(sampled_latents_tensor)

print("Sampled Vectors Shape:", sampled_latents_tensor.shape)  # Should be (5, *) depending on your data shape
print("Random Vectors Shape:", random_latents_tensor.shape)  # Should be (5, *) depending on your data shape

sampled_test_images = decoder(sampled_latents_tensor).view(-1, 1, 28, 28)
random_test_images = decoder(random_latents_tensor).view(-1, 1, 28, 28)

print("Sampled Images Shape:", sampled_test_images.shape)  # Should be (5, *) depending on your data shape
utils.save_images(sampled_test_images, "sampled_test_images.png")
utils.save_images(random_test_images, "random_test_images.png")

Sampled Vectors Shape: torch.Size([5, 256])
Random Vectors Shape: torch.Size([5, 256])
Sampled Images Shape: torch.Size([5, 1, 28, 28])
